In [ ]:
import os
import sys
sys.path.append("../../")

import pandas as pd
import numpy as np

import torch
import matplotlib.pyplot as plt

from copy import deepcopy

from analysis.Inference.utils import load_all_models, load_all_client_loader
from probutils import vizualize_cka_model
from cka import CKACalculator

import torch.nn as nn

import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# {model_type : [best_model wandb id, best_model round]}
model_type_dict = {"Center": ['sinister-pulse-2', -1], # No need to specify epoch
                   "Local": ["lurking-apparition-2", -1], # No need to specify epoch
                   "FedAvg" : ["silvery-dragon-1", -1],
                   "FedProx": ["glorious-mountain-3", -1],  
                   "MOON": ["fancy-frog-10", -1]
                   }
BASE = "/NFS/Users/moonsh/AdaptFL/ckpt/"
device = torch.device("cuda:0")
batch_size, worker = 66, 0

In [ ]:
model_dict, config = load_all_models(model_type_dict, BASE, device, batch_size, worker)

config.data_path = '/NFS/Users/moonsh/data/FLData/'

(train_client_list, 
 val_client_list, 
 test_client_list) = load_all_client_loader(config, _mode='all')

savepath = '/NFS/Users/moonsh/AdaptFL/result/'

In [ ]:
concat_client_list = []

for val, test in zip(val_client_list, test_client_list):
    concat_dataset = val.dataset+test.dataset
    concat_loader = torch.utils.data.DataLoader(concat_dataset, batch_size=batch_size, shuffle=False)
    concat_client_list.append(concat_loader)

print(np.array(concat_client_list).shape)

In [ ]:
vizualize_cka_model(model_dict,
                    test_client_list,
                    device=device,
                    criterion='Local',)

In [ ]:
vizualize_cka_model(model_dict,
                    test_client_list,
                    device=device,
                    criterion='Center',)

## CKA Calculator Demo

In [ ]:
# temp_loader = val_client_list[0]

# calculator = CKACalculator(model1=model_dict['Center'].to(device),
#                            model2=model_dict['FedAvg'].to(device),
#                            dataloader=temp_loader,
#                            hook_layer_types=(nn.Conv3d, nn.Conv2d), # dummy_val: nn.Conv2d
#                            num_epochs=1,
#                            epsilon=1e-10)

# cka_output = calculator.calculate_cka_matrix().cpu().detach().numpy()
# cka_output = np.diag(cka_output)

In [ ]:
# # Extract the layer names
# except_layer = []
# print(len(calculator.module_names_X))
# for i, name in enumerate(calculator.module_names_X):
#     if 'down'in name:
#         except_layer.append(i)
#     # print(f"{name}")

In [ ]:
# cka_output = np.delete(cka_output, except_layer)